This file is a snapshot of our struggles with NetDiffusion. Some cells contain some AI-assisted code. This file does not capture the evolution over time, but simply the final state.

In [ ]:
# %cd /content/drive/MyDrive/SCADA_NetDiff/
%cd /content

/content


In [ ]:
!rm -rf sample_data/ color_corrected_generated_traffic_images/ generated_nprint/ generated_traffic_images/ nprint_traffic_images/ nprint_traffic/ pcaps/ replayable_generated_nprints/ replayable_generated_pcaps/ trained-sd3-lora-miniature/ wandb/ sample_embeddings.parquet nprint-1.2.1/ scripts/ nprint-1.2.1.tar.gz* payload_mask.png column_order_*.txt conditioning*

In [ ]:
!pip install -q -U git+https://github.com/huggingface/diffusers

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
!pip install -q -U \
    transformers \
    accelerate \
    wandb \
    bitsandbytes \
    peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.9/22.9 MB 121.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 45.9 MB/s eta 0:00:00


In [ ]:
!pip install pandas torchvision pyarrow sentencepiece controlnet_aux scapy gdown opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 67.3 MB/s eta 0:00:00


In [ ]:
!git clone https://github.com/noise-lab/NetDiffusion.git
!mv NetDiffusion/scripts scripts && rm -rf NetDiffusion/

Cloning into 'NetDiffusion'...
remote: Enumerating objects: 179, done.
remote: Counting objects: 100% (179/179), done.
remote: Compressing objects: 100% (115/115), done.
remote: Total 179 (delta 81), reused 123 (delta 49), pack-reused 0 (from 0)
Receiving objects: 100% (179/179), 5.00 MiB | 21.07 MiB/s, done.
Resolving deltas: 100% (81/81), done.


As SD3 is gated, before using it with diffusers you first need to go to the Stable Diffusion 3 Medium Hugging Face page, fill in the form and accept the gate. Once you are in, you need to log in so that your system knows you’ve accepted the gate. Use the command below to log in:
#Ignore if already logged in.

## Hugging Face Authentication

As SD3 is gated, you need to:
1. Go to the [Stable Diffusion 3 Medium Hugging Face page](https://huggingface.co/stabilityai/stable-diffusion-3-medium-diffusers)
2. Fill in the form and accept the gate
3. Log in to authenticate your access

Check if you're already logged in first:

In [ ]:
# If not logged in, run this command and enter your token when prompted
!hf auth login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
/usr/lib/python3.12/getpass.py:91: GetPassWarning: Can not control echo on the terminal.
  passwd = fallback_getpass(prompt, stream)
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Token is valid (permission: write).
The token `scada-generator` has been saved to /root/.cache/huggingface

In [ ]:
!sudo apt-get install libpcap-dev
!wget https://github.com/nprint/nprint/releases/download/v1.2.1/nprint-1.2.1.tar.gz
!tar -xvf nprint-1.2.1.tar.gz
!cd nprint-1.2.1 && ./configure && make && make install

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libdbus-1-dev libpcap0.8 libpcap0.8-dev
The following NEW packages will be installed:
  libdbus-1-dev libpcap-dev libpcap0.8 libpcap0.8-dev
0 upgraded, 4 newly installed, 0 to remove and 41 not upgraded.
Need to get 607 kB of archives.
After this operation, 2,238 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libpcap0.8 amd64 1.10.1-4ubuntu1.22.04.1 [145 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libdbus-1-dev amd64 1.12.20-2ubuntu4.1 [188 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libpcap0.8-dev amd64 1.10.1-4ubuntu1.22.04.1 [270 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libpcap-dev amd64 1.10.1-4ubuntu1.22.04.1 [3,326 B]
Fetched 607 kB in 2s (289 kB/s)
debconf: unable to initialize frontend: Dialog
deb

CHANGE DOWNLOADED PCAP BELOW

In [ ]:
!mkdir pcaps
!mkdir nprint_traffic
# Modbus messaging
!cd pcaps && wget "https://drive.google.com/uc?export=download&id=1rePKAWeVPMIN21WPcF34g3WSfzLi9mCm" -O input_traces.pcapng

# SEL
# REDACTED SEL LINK DUE TO CLOSED-SOURCE DATASET

# !cd pcaps && wget https://github.com/antoine-lemay/Modbus_dataset/raw/master/channel_2d_3s.zip
# !cd pcaps && unzip channel_2d_3s.zip

--2025-12-04 05:58:36--  https://drive.google.com/uc?export=download&id=1VwIHDyfEU-NFjTTj-gZ7TPYoezoqN2br
Resolving drive.google.com (drive.google.com)... 142.251.10.139, 142.251.10.100, 142.251.10.101, ...
Connecting to drive.google.com (drive.google.com)|142.251.10.139|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1VwIHDyfEU-NFjTTj-gZ7TPYoezoqN2br&export=download [following]
--2025-12-04 05:58:36--  https://drive.usercontent.google.com/download?id=1VwIHDyfEU-NFjTTj-gZ7TPYoezoqN2br&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 172.253.118.132, 2404:6800:4003:c05::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|172.253.118.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1907721 (1.8M) [application/octet-stream]
Saving to: ‘input_traces.pcapng’

input_traces.pcapng 100%[===================>]   1.82M  -

In [ ]:
# !nprint -F -1 -P pcaps/input_traces.pcapng -4 -i -6 -t -u -p 0 -c 1024 -W nprint_traffic/modbus.nprint
# p = 8, include 8 payload bytes to stay divisible by 64
!nprint -P pcaps/input_traces.pcapng -4 -6 -t -u -i -p 8 -c 1024 -W nprint_traffic/modbus.nprint

## Convert nPrint to PNG Images

This script transforms `.nprint` files—tabular feature representations of network packets—into fixed-size PNG images that can be used as input for SD fine-tuning.

**Input:** `.nprint` files generated from packet captures (PCAPs) using the nPrint tool. Each file is a CSV-like matrix where each row represents a single packet and columns correspond to extracted features.

**Preprocessing:**
- Drops IP address-related columns to avoid injecting identifiable or non-generalizable information into the model
- Maps integer values in the remaining columns to RGBA color tuples to visualize numeric features as colored pixels

**Padding:** Pads each image to a uniform height (default 1024) using a solid background to ensure model input consistency across varying packet counts.

**Output:** Saves a PNG file for each `.nprint` file, preserving the packet structure as a vertically stacked color-coded image (rows = packets, cols = features).

In [ ]:
!python ./scripts/nprint_to_png.py -i ./nprint_traffic/ -o ./nprint_traffic_images

Processing modbus.nprint
/content/./scripts/nprint_to_png.py:26: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  np_df = np.array(df.applymap(np.array).to_numpy().tolist())
/content/./scripts/nprint_to_png.py:29: DeprecationWarning: 'mode' parameter is deprecated and will be removed in Pillow 13 (2026-10-15)
  img = Image.fromarray(np_img, 'RGBA')


## Compute Embeddings

Generates text prompt embeddings via a Stable Diffusion 3 pipeline and T5 text encoder. Maps each local PNG image to a unique SHA-256 hash and associates it with the computed embeddings. Stores the resulting image-hash-to-embedding data in a .parquet file for further processing.

Here we are using the default instance prompt "pixelated network data for type-0 application traffic". You can configure this by referring to the compute_embeddings.py script for details on other supported arguments.

In [ ]:
!python ./scripts/compute_embeddings.py

2025-12-04 05:58:54.200425: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-04 05:58:54.217913: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764827934.239100    2168 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764827934.245768    2168 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1764827934.261966    2168 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

Compute embeddings:
* Generates text prompt embeddings via a Stable Diffusion 3 pipeline and T5 text encoder.
* Maps each local PNG image to a unique SHA-256 hash and associates it with the computed embeddings.
* Stores the resulting image-hash-to-embedding data in a .parquet file for further processing.
* Here we are using the default instance prompt "pixelated network data for type-0 application traffic".
* But you can configure this. Refer to the compute_embeddings.py script for details on other supported arguments.


## Train LoRA Adapter on Stable Diffusion 3 (Miniature Setup)

This command launches training using `accelerate` with DreamBooth-style LoRA tuning, optimized for quick experimentation or demo runs.

**⚠️ Current configuration uses:**
- Only 1 training step
- Small batch size
- No warmup
- High learning rate

**Intended only for testing or verifying training scripts**, NOT for quality results. For actual training, increase `max_train_steps`, adjust learning rate, and consider enabling full validation and saving checkpoints.

In [ ]:
!export WANDB_MODE=offline && accelerate launch ./scripts/train_dreambooth_lora_sd3_miniature.py \
  --pretrained_model_name_or_path="stabilityai/stable-diffusion-3-medium-diffusers" \
  --instance_data_dir="nprint_traffic_images" \
  --data_df_path="sample_embeddings.parquet" \
  --output_dir="trained-sd3-lora-miniature" \
  --mixed_precision="fp16" \
  --instance_prompt="pixelated network data for type-0 application traffic" \
  --train_batch_size=2 \
  --gradient_accumulation_steps=4 \
  --gradient_checkpointing \
  --use_8bit_adam \
  --learning_rate=1e-4 \
  --report_to="wandb" \
  --lr_scheduler="cosine" \
  --lr_warmup_steps=500 \
  --max_train_steps=50 \
  --seed="0" \
  --validation_prompt="pixelated network data for type-0 application traffic" \
  --validation_epochs=100

The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_processes` was set to a value of `1`
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
2025-12-04 06:00:43.740360: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-04 06:00:43.758662: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764828043.780819    2810 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register 

## Inference Pipeline for SD3 + ControlNet + LoRA

This cell demonstrates the full generation process:
- Loads Stable Diffusion 3 Medium base model
- Loads ControlNet (Canny edge-based guidance)
- Loads LoRA weights fine-tuned on pixelated network traffic
- Applies edge-conditioned generation using a sample input

In [ ]:
"""
Create a reference nprint file with dynamic masking.
- Columns where ALL values are -1 stay masked
- Columns with any non-(-1) values are preserved
- Outputs an nprint CSV file for use with reconstruction
"""

import pandas as pd

nprint_path = "./nprint_traffic/modbus.nprint"
output_nprint_path = "./conditioning.nprint"

# Load the nprint file
df = pd.read_csv(nprint_path)

# Drop the src_ip column (it's handled separately in reconstruction)
if 'src_ip' in df.columns:
    df = df.drop(columns=['src_ip'])

print(f"Original dataframe: {len(df.columns)} columns, {len(df)} rows")

# === DYNAMIC MASKING: Find columns where ALL values are -1 ===
cols_to_mask = [col for col in df.columns if (df[col] == -1).all()]
cols_to_keep = [col for col in df.columns if col not in cols_to_mask]

print(f"Columns to MASK (all -1s): {len(cols_to_mask)}")
print(f"Columns to KEEP: {len(cols_to_keep)}")

# Show which protocol groups are being masked
protocol_groups = {}
for col in cols_to_mask:
    prefix = col.split('_')[0]
    protocol_groups[prefix] = protocol_groups.get(prefix, 0) + 1

print(f"Masked protocol groups: {protocol_groups}")

# Create the reference nprint (masked columns stay -1, kept columns have data)
reference_df = df.copy()

# Columns already have correct values - masked ones are already -1
# Just verify and save
print(f"✓ Masked {len(cols_to_mask)} columns (all -1)")
print(f"✓ Preserved {len(cols_to_keep)} columns with data")

# Save the reference nprint
reference_df.to_csv(output_nprint_path, index=False)
print(f"✓ Saved reference nprint to {output_nprint_path}")
print(f"✓ Shape: {reference_df.shape[0]} rows x {reference_df.shape[1]} columns")

# Also save column order for reference
with open("./column_order.txt", "w") as f:
    for col in reference_df.columns:
        f.write(f"{col}\n")
print(f"✓ Saved column order to ./column_order.txt")

Original dataframe: 1472 columns, 1024 rows
Columns to MASK (all -1s): 992
Columns to KEEP: 480
Masked protocol groups: {'ipv4': 320, 'ipv6': 320, 'tcp': 224, 'udp': 64, 'icmp': 64}
✓ Masked 992 columns (all -1)
✓ Preserved 480 columns with data
✓ Saved reference nprint to ./conditioning.nprint
✓ Shape: 1024 rows x 1472 columns
✓ Saved column order to ./column_order.txt


In [ ]:
!python ./scripts/nprint_to_png.py -i . -o .

Processing conditioning.nprint
/content/./scripts/nprint_to_png.py:26: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  np_df = np.array(df.applymap(np.array).to_numpy().tolist())
/content/./scripts/nprint_to_png.py:29: DeprecationWarning: 'mode' parameter is deprecated and will be removed in Pillow 13 (2026-10-15)
  img = Image.fromarray(np_img, 'RGBA')


## Post-Generation Processing Pipeline

This cell performs a 3-stage transformation of the generated PNG images:

1. **Color Augmentation:** Applies standardized color shifts to improve nprint reconstruction accuracy
2. **Image-to-nPrint Conversion:** Converts augmented images back into nPrint-compatible feature format using a reference file to maintain consistent structure
3. **Heuristic Correction & PCAP Reconstruction:** Reconstructs valid and replayable PCAP files from the diffusion-generated nPrint representation

This pipeline enables turning synthetic traffic images back into replayable network traffic for evaluation or simulation.

Final PCAP files are stored in `replayable_generated_pcaps`

In [ ]:
# -----------------------------------------------------------
# 🎯 Inference Pipeline for SD3 + ControlNet + LoRA
#
# This cell demonstrates the full generation process:
#  - Loads Stable Diffusion 3 Medium base model
#  - Loads ControlNet (Canny edge-based guidance)
#  - Loads LoRA weights fine-tuned on pixelated network traffic
#  - Applies edge-conditioned generation using a sample input
#
# -----------------------------------------------------------
# flush()
import os
import torch
import cv2
from PIL import Image
from diffusers import StableDiffusion3ControlNetPipeline, SD3ControlNetModel
from diffusers.utils import load_image

# Make sure our output folder exists
os.makedirs("generated_traffic_images", exist_ok=True)

# Base SD 3.0 model
base_model_path = "stabilityai/stable-diffusion-3-medium-diffusers"

# Canny-based ControlNet
controlnet_path = "InstantX/SD3-Controlnet-Canny"

# Load the ControlNet and pipeline
controlnet = SD3ControlNetModel.from_pretrained(
    controlnet_path, torch_dtype=torch.float16
)
pipe = StableDiffusion3ControlNetPipeline.from_pretrained(
    base_model_path,
    controlnet=controlnet,
)

# Load LoRA weights
lora_output_path = "trained-sd3-lora-miniature"
pipe.load_lora_weights(lora_output_path)

# Move pipeline to GPU (half precision)
pipe.to("cuda", torch.float16)
pipe.enable_sequential_cpu_offload()

# ----------------------------------------------------
# 1) Convert original control image to Canny edges via OpenCV
# ----------------------------------------------------
orig_path = "./conditioning.png" #"./scripts/traffic_conditioning_image.png"
orig_bgr = cv2.imread(orig_path)

if orig_bgr is None:
    raise ValueError(f"Could not load file: {orig_path}")

# Convert to grayscale
gray = cv2.cvtColor(orig_bgr, cv2.COLOR_BGR2GRAY)

# Generate Canny edge map (tweak thresholds as needed)
edges = cv2.Canny(gray, 100, 200)

# Convert single-channel edge map to 3-channel RGB
edges_rgb = cv2.cvtColor(edges, cv2.COLOR_GRAY2RGB)

# Convert to PIL for use in ControlNet pipeline
control_image = Image.fromarray(edges_rgb)
orig_width, orig_height = control_image.size

# Determine target dimensions that are multiples of 64
target_height = 1024 # 128 * 8
target_width = 1472


if orig_width != target_width or orig_height != target_height:
    print(f"Resizing control image from ({orig_width}, {orig_height}) to ({target_width}, {target_height})")
    control_image = control_image.resize((target_width, target_height))

# ----------------------------------------------------
# 3) Set up prompts and run pipeline
# ----------------------------------------------------
prompt = "pixelated network data for type-0 application traffic"
generator = torch.manual_seed(0)  # reproducibility

# Generate at 1024x1472 to match the new control image and model requirements
image = pipe(
    prompt=prompt,
    num_inference_steps=20,
    generator=generator,
    height=target_height,
    width=target_width,
    control_image=control_image,
    controlnet_conditioning_scale=0.8,  # increase to adhere more strongly to edges
).images[0]

# ----------------------------------------------------
# 4) Save the generated image
# ----------------------------------------------------
output_path = os.path.join("generated_traffic_images", "generated_traffic.png")
image.save(output_path)
print(f"Generated image saved to: {output_path}")

Flax classes are deprecated and will be removed in Diffusers v1.0.0. We recommend migrating to PyTorch classes or pinning your version of Diffusers.
Flax classes are deprecated and will be removed in Diffusers v1.0.0. We recommend migrating to PyTorch classes or pinning your version of Diffusers.


config.json:   0%|          | 0.00/397 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/1.19G [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/9 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers
No LoRA keys associated to CLIPTextModelWithProjection found with the prefix='text_encoder'. This is safe to ignore if LoRA state dict didn't originally have any CLIPTextModelWithProjection related params. You can also try specifying `prefix=None` to resolve the warning. Otherwise, open an issue if you think it's unexpected: https://github.com/huggingface/diffusers/issues/new
No LoRA keys associated to CLIPTextModelWithProjection found with the prefix='text_encoder_2'. This is safe to ignore if LoRA state dict didn't originally have any CLIPTextModelWithProjection related params. You can also try specifying `prefix=None` to resolve the warning. Otherwise, open an issue if you think it's unexpected: https://github.com/huggingface/diffusers/issues/new


Resizing control image from (1152, 1024) to (1472, 1024)


  0%|          | 0/20 [00:00<?, ?it/s]

Generated image saved to: generated_traffic_images/generated_traffic.png


In [ ]:
"""
Add 64 payload_bit columns (payload_bit_0 through payload_bit_63)
with all values set to 0 to the nprint file
"""
import pandas as pd

def add_payload_columns(input_path, output_path, num_payload_bits=64):
    """
    Add payload_bit_0 through payload_bit_{num_payload_bits-1} columns
    with all values set to 0.

    Args:
        input_path: Path to input nprint file
        output_path: Path to save output file with payload columns
        num_payload_bits: Number of payload bit columns to add (default: 64)
    """
    print(f"Loading {input_path}...")
    df = pd.read_csv(input_path)

    original_cols = len(df.columns)
    original_rows = len(df)
    print(f"  Original: {original_rows} rows × {original_cols} columns")

    # Create payload column names
    payload_cols = [f'payload_bit_{i}' for i in range(num_payload_bits)]

    # Add each payload column with all zeros
    for col in payload_cols:
        df[col] = 0

    # Save to output
    df.to_csv(output_path, index=False)

    new_cols = len(df.columns)
    print(f"  ✓ Added {num_payload_bits} payload columns")
    print(f"  New: {original_rows} rows × {new_cols} columns")
    print(f"  ✓ Saved to {output_path}")

    # Show the last few column names to verify
    print(f"\n  Last 5 columns: {list(df.columns[-5:])}")

# Usage example
add_payload_columns(
    input_path="./scripts/correct_format.nprint",
    output_path="./scripts/correct_format.nprint",
    num_payload_bits=64
)

Loading ./scripts/correct_format.nprint...
  Original: 1024 rows × 1409 columns
  ✓ Added 64 payload columns
  New: 1024 rows × 1473 columns
  ✓ Saved to ./scripts/correct_format.nprint

  Last 5 columns: ['payload_bit_59', 'payload_bit_60', 'payload_bit_61', 'payload_bit_62', 'payload_bit_63']


In [ ]:
"""
Remove src_ip and save it separately for reconstruction,
then save the processed file WITH index
"""
import pandas as pd

def remove_src_ip_with_save(input_path, output_path, src_ip_save_path):
    """
    Remove src_ip, save it separately, and save remaining data with index.
    """
    print(f"Loading {input_path}...")
    df = pd.read_csv(input_path)

    original_cols = len(df.columns)
    print(f"  Original columns: {original_cols}, Rows: {len(df)}")

    # Save src_ip data
    if 'src_ip' in df.columns:
        src_ip_df = df[['src_ip']].copy()
        src_ip_df.to_csv(src_ip_save_path, index=True)
        print(f"  ✓ Saved src_ip to {src_ip_save_path}")

        df = df.drop(columns=['src_ip'])
        print(f"  ✓ Removed 'src_ip' column")

    # Save WITH index=True (adds row numbers as first column)
    df.to_csv(output_path, index=True)

    print(f"  New data columns: {len(df.columns)}")
    print(f"  Total columns in file (with index): {len(df.columns) + 1}")
    print(f"  Saved to {output_path}\n")

# Usage
remove_src_ip_with_save(
    input_path="./nprint_traffic/modbus.nprint",
    output_path="./scripts/column_example.nprint",
    src_ip_save_path="./scripts/saved_src_ip_reference.csv"
)

Loading ./nprint_traffic/modbus.nprint...
  Original columns: 1473, Rows: 1024
  ✓ Saved src_ip to ./scripts/saved_src_ip_reference.csv
  ✓ Removed 'src_ip' column
  New data columns: 1472
  Total columns in file (with index): 1473
  Saved to ./scripts/column_example.nprint



In [ ]:
# -----------------------------------------------------------
# 🔄 Post-Generation Processing Pipeline
#
# This cell performs a 3-stage transformation of the generated PNG images:
#   1. Applies color correction for standardization.
#   2. Converts augmented images back into nPrint-compatible feature format.
#   3. Applies heuristic corrections and reconstructs valid PCAP files.
#
# ⚙️ This pipeline enables turning synthetic traffic images
#    back into replayable network traffic for evaluation or simulation.
# -----------------------------------------------------------

# 🎨 Step 1: Color Augmentation
# Applies standardized color shifts to improve nprint reconstruction accuracy
!python ./scripts/color_processor.py \
  --input_dir="./generated_traffic_images" \
  --output_dir="./color_corrected_generated_traffic_images"
# -----------------------------------------------------------
# 🔁 Step 2: Image-to-nPrint Conversion
# Converts augmented PNG images back into `.nprint` tabular format.
#
# Uses a reference `.nprint` file to maintain consistent structure and column order.
# This step allows diffusion-generated visual traffic to be fed into analysis tools.
# -----------------------------------------------------------
!python ./scripts/image_to_nprint.py \
  --org_nprint ./scripts/column_example.nprint \
  --input_dir ./color_corrected_generated_traffic_images \
  --output_dir ./generated_nprint

Processed 1 images.
Processing ./color_corrected_generated_traffic_images/generated_traffic.png with size 1472 x 1024
Saved ./generated_nprint/generated_traffic.nprint
Done! Converted 1 .png files into .nprint format.


In [ ]:
# -----------------------------------------------------------
# 🧠 Step 3: Heuristic Correction & PCAP Reconstruction
#
# This step reconstructs a valid and replayable `.pcap` file
# from the diffusion-generated `.nprint` representation.
# 🔍 Core Functionalities:
# ✅ Intra-packet corrections (fixes within individual packets).
# 🔁 Inter-packet dependency enforcement.
# 🔧 Reconstruction:
#   - Save the corrected `.nprint` to disk
#   - Call `nprint -W` to convert `.nprint` into `.pcap` using external tool
#   - Run Scapy-based checksum updates to ensure IPv4 validity
#   - Reconvert final `.pcap` back to `.nprint` (with fixed layout) for downstream tasks
# -----------------------------------------------------------
!python ./scripts/mass_reconstruction.py \
  --input_dir ./generated_nprint \
  --output_pcap_dir ./replayable_generated_pcaps \
  --output_nprint_dir ./replayable_generated_nprints \
  --formatted_nprint_path ./scripts/correct_format.nprint
# Final Pcap is stored in replayable_generated_pcaps


Processing: ./generated_nprint/generated_traffic_with_src_ip.nprint -> ./replayable_generated_pcaps/generated_traffic_with_src_ip.pcap
Running command:
  python3 ./scripts/reconstruction.py --generated_nprint_path './generated_nprint/generated_traffic_with_src_ip.nprint' --formatted_nprint_path './scripts/correct_format.nprint' --output './replayable_generated_pcaps/generated_traffic_with_src_ip.pcap' --nprint './replayable_generated_nprints/generated_traffic_with_src_ip.nprint'
tcp
malloc(): invalid next size (unsorted)
Aborted (core dumped)
Success! Created ./replayable_generated_pcaps/generated_traffic_with_src_ip.pcap and possibly updated ./replayable_generated_nprints/generated_traffic_with_src_ip.nprint.

Processing: ./generated_nprint/generated_traffic.nprint -> ./replayable_generated_pcaps/generated_traffic.pcap
Running command:
  python3 ./scripts/reconstruction.py --generated_nprint_path './generated_nprint/generated_traffic.nprint' --formatted_nprint_path './scripts/correct

In [ ]:
# For long running jobs, copy output to Drive
!cp -r nprint_traffic_images drive/MyDrive/SCADA_NetDiff
!cp -r color_corrected_generated_traffic_images drive/MyDrive/SCADA_NetDiff
!cp -r replayable_generated_pcaps drive/MyDrive/SCADA_NetDiff
!cp -r replayable_generated_nprints drive/MyDrive/SCADA_NetDiff
!cp -r trained-sd3-lora-miniature drive/MyDrive/SCADA_NetDiff

cp: cannot create directory 'drive/MyDrive/SCADA_NetDiff': No such file or directory
cp: cannot create directory 'drive/MyDrive/SCADA_NetDiff': No such file or directory
cp: cannot create directory 'drive/MyDrive/SCADA_NetDiff': No such file or directory
cp: cannot create directory 'drive/MyDrive/SCADA_NetDiff': No such file or directory
cp: cannot create directory 'drive/MyDrive/SCADA_NetDiff': No such file or directory
